In [ ]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt
from google.colab import drive, files
from google.colab.output import eval_js
from IPython.display import display, Javascript, clear_output
from base64 import b64decode
import cv2
import os
import shutil
import urllib.request
from datetime import datetime

# --- 1. CONFIGURATION ---
# 📂 Where models live
MODEL_FOLDER = "/content/drive/MyDrive/Dog_Project_Models"

# 📂 Where to save user photos (Privacy: Set to None or "" to disable saving)
HISTORY_FOLDER = "/content/drive/MyDrive/Dog_App_History"

# --- 2. MOUNT & LOAD ---
print("⏳ Mounting Drive & Loading Models...")
drive.mount('/content/drive')

if HISTORY_FOLDER and not os.path.exists(HISTORY_FOLDER):
    os.makedirs(HISTORY_FOLDER)

try:
    models = {
        'resnet50': keras.models.load_model(f"{MODEL_FOLDER}/resnet50_dogs.keras"),
        'efficientnetb0': keras.models.load_model(f"{MODEL_FOLDER}/efficientnetb0_dogs.keras"),
        'mobilenetv2': keras.models.load_model(f"{MODEL_FOLDER}/mobilenetv2_dogs.keras")
    }
    mlp = keras.models.load_model(f"{MODEL_FOLDER}/mlp_ensemble.keras")
    print("✅ AI Models Loaded!")
except OSError:
    print("❌ Error: Could not find models. Please check 'Dog_Project_Models' in Drive.")

# --- 3. DOWNLOAD OPENCV ---
print("🔄 Initializing Smart Crop...")
files_needed = {
    "MobileNetSSD_deploy.prototxt.txt": "https://raw.githubusercontent.com/djmv/MobilNet_SSD_opencv/master/MobileNetSSD_deploy.prototxt",
    "MobileNetSSD_deploy.caffemodel": "https://raw.githubusercontent.com/djmv/MobilNet_SSD_opencv/master/MobileNetSSD_deploy.caffemodel"
}

for fname, url in files_needed.items():
    if not os.path.exists(fname):
        try: urllib.request.urlretrieve(url, fname)
        except: pass

try:
    net = cv2.dnn.readNetFromCaffe("MobileNetSSD_deploy.prototxt.txt", "MobileNetSSD_deploy.caffemodel")
    print("✅ Smart Crop Ready!")
except:
    net = None
    print("⚠️ Smart Crop Disabled")

# --- 4. BREED DATA ---
breed_info = {
    'american_bulldog': {'life': '10-12 years', 'traits': 'Confident, Social, Active'},
    'american_pit_bull_terrier': {'life': '8-15 years', 'traits': 'Loyal, Courageous, Friendly'},
    'basset_hound': {'life': '12-13 years', 'traits': 'Patient, Low-energy, Charming'},
    'beagle': {'life': '10-15 years', 'traits': 'Curious, Merry, Friendly'},
    'boxer': {'life': '10-12 years', 'traits': 'Bright, Fun-loving, Active'},
    'chihuahua': {'life': '14-16 years', 'traits': 'Charming, Graceful, Sassy'},
    'english_cocker_spaniel': {'life': '12-14 years', 'traits': 'Merry, Responsive, Gentle'},
    'english_setter': {'life': '12 years', 'traits': 'Friendly, Mellow, Merry'},
    'german_shorthaired': {'life': '10-12 years', 'traits': 'Friendly, Smart, Willing'},
    'great_pyrenees': {'life': '10-12 years', 'traits': 'Smart, Patient, Calm'},
    'havanese': {'life': '14-16 years', 'traits': 'Funny, Intelligent, Outgoing'},
    'japanese_chin': {'life': '10-12 years', 'traits': 'Charming, Noble, Loving'},
    'keeshond': {'life': '12-15 years', 'traits': 'Friendly, Lively, Outgoing'},
    'leonberger': {'life': '7 years', 'traits': 'Gentle, Friendly, Playful'},
    'miniature_pinscher': {'life': '12-16 years', 'traits': 'Fearless, Spirited, Proud'},
    'newfoundland': {'life': '9-10 years', 'traits': 'Sweet, Patient, Devoted'},
    'pomeranian': {'life': '12-16 years', 'traits': 'Lively, Bold, Inquisitive'},
    'pug': {'life': '13-15 years', 'traits': 'Loving, Charming, Mischievous'},
    'saint_bernard': {'life': '8-10 years', 'traits': 'Playful, Charming, Inquisitive'},
    'samoyed': {'life': '12-14 years', 'traits': 'Adaptable, Friendly, Gentle'},
    'scottish_terrier': {'life': '12 years', 'traits': 'Independent, Confident, Spirited'},
    'shiba_inu': {'life': '13-16 years', 'traits': 'Alert, Active, Attentive'},
    'staffordshire_bull_terrier': {'life': '12-14 years', 'traits': 'Clever, Brave, Tenacious'},
    'wheaten_terrier': {'life': '12-14 years', 'traits': 'Happy, Steady, Self-Confident'},
    'yorkshire_terrier': {'life': '11-15 years', 'traits': 'Sprightly, Tomboyish, Affectionate'}
}

cat_breeds = {"Abyssinian","Bengal","Birman","Bombay","British_Shorthair","Egyptian_Mau",
              "Maine_Coon","Persian","Ragdoll","Russian_Blue","Siamese","Sphynx"}
info = tfds.builder("oxford_iiit_pet").info
dog_names = [n for n in info.features["label"].names if n not in cat_breeds]

# --- 5. HELPER FUNCTIONS ---
def take_photo(filename='photo.jpg', quality=0.8):
    js = Javascript('''
    async function takePhoto(quality) {
      const div = document.createElement('div');
      const capture = document.createElement('button');
      capture.textContent = 'Capture';
      div.appendChild(capture);
      const video = document.createElement('video');
      video.style.display = 'block';
      const stream = await navigator.mediaDevices.getUserMedia({video: true});
      document.body.appendChild(div);
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);
      await new Promise((resolve) => capture.onclick = resolve);
      const canvas = document.createElement('canvas');
      canvas.width = video.videoWidth;
      canvas.height = video.videoHeight;
      canvas.getContext('2d').drawImage(video, 0, 0);
      stream.getVideoTracks()[0].stop();
      div.remove();
      return canvas.toDataURL('image/jpeg', quality);
    }
    ''')
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])
    with open(filename, 'wb') as f: f.write(binary)
    return filename

def get_dog_roi(img_path):
    img = cv2.imread(img_path)
    if img is None: return None, False
    if net is None: return cv2.cvtColor(img, cv2.COLOR_BGR2RGB), False

    (h, w) = img.shape[:2]
    blob = cv2.dnn.blobFromImage(cv2.resize(img, (300, 300)), 0.007843, (300, 300), 127.5)
    net.setInput(blob)
    detections = net.forward()

    best_conf = 0.0
    dog_box = None
    for i in range(detections.shape[2]):
        conf = detections[0, 0, i, 2]
        if int(detections[0, 0, i, 1]) == 12 and conf > 0.2:
            if conf > best_conf:
                best_conf = conf
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                dog_box = box.astype("int")
    if dog_box is not None:
        (sx, sy, ex, ey) = dog_box
        sx, sy = max(0, sx-20), max(0, sy-20)
        ex, ey = min(w, ex+20), min(h, ey+20)
        return cv2.cvtColor(img[sy:ey, sx:ex], cv2.COLOR_BGR2RGB), True
    return cv2.cvtColor(img, cv2.COLOR_BGR2RGB), False

def analyze_image(filename, source="upload"):
    roi, cropped = get_dog_roi(filename)
    if roi is None: return

    # --- PRIVACY: SAVE TO DRIVE (Optional) ---
    if HISTORY_FOLDER:
        ts = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
        save_path = f"{HISTORY_FOLDER}/Dog_{source}_{ts}.jpg"
        shutil.copy(filename, save_path)
        print(f"💾 Saved to Drive: {save_path}")
    # -----------------------------------------

    if not cropped:
        print("⚠️ No dog detected (Smart Crop inactive).")

    # Predict
    img_r = cv2.resize(roi, (224, 224))
    img_arr = np.expand_dims(img_r, axis=0) / 255.0

    p1 = models['resnet50'].predict(img_arr, verbose=0)
    p2 = models['efficientnetb0'].predict(img_arr, verbose=0)
    p3 = models['mobilenetv2'].predict(img_arr, verbose=0)
    final = mlp.predict(np.concatenate([p1, p2, p3], axis=1), verbose=0)[0]

    # Top 3
    top_3 = np.argsort(final)[-3:][::-1]
    top_name = dog_names[top_3[0]]
    top_conf = final[top_3[0]] * 100
    info = breed_info.get(top_name, {'life': '?', 'traits': '?'})

    plt.figure(figsize=(6,6))
    plt.imshow(roi)
    plt.title(f"🏆 {top_name} ({top_conf:.1f}%)\n{info['life']}", color="green")
    plt.axis("off")
    plt.show()

    print(f"\n📊 TOP 3 PREDICTIONS:")
    for i in top_3:
        print(f"• {dog_names[i]}: {final[i]*100:.1f}%")

    if os.path.exists(filename): os.remove(filename)

# --- 6. RUN MENU ---
def run_cloud_app():
    print("🐶 CLOUD DOG CLASSIFIER")
    print("1️⃣. Use Camera")
    print("2️⃣. Upload File")
    c = input("👉 Choose 1 or 2: ")

    if c == '1':
        print("📸 Get ready...")
        fname = take_photo('temp_cam.jpg')
        if fname: analyze_image(fname, "camera")
    elif c == '2':
        uploaded = files.upload()
        for f in uploaded.keys():
            analyze_image(f, "upload")
    else:
        print("❌ Invalid choice")

run_cloud_app()

In [ ]:
import shutil
from google.colab import files
from google.colab import drive
import os

# 1. Mount Drive to find the files
drive.mount('/content/drive')
source_folder = "/content/drive/MyDrive/Dog_Project_Models"

# 2. Check if files exist
if os.path.exists(source_folder):
    print("⏳ Zipping your models... (This might take a minute)")

    # 3. Create a ZIP file of the entire folder
    shutil.make_archive("/content/Dog_Models_Backup", 'zip', source_folder)

    # 4. Trigger the Download
    print("⬇️ Download starting! Check your browser's download bar.")
    files.download("/content/Dog_Models_Backup.zip")
else:
    print("⚠️ Error: Could not find the 'Dog_Project_Models' folder in your Drive.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
⏳ Zipping your models... (This might take a minute)
⬇️ Download starting! Check your browser's download bar.


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>